In [1]:
getwd()

[1] "G:/zll/htseq_out"

In [2]:
library("DESeq")

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from 'package:stats':

    xtabs

The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, intersect,
    is.unsorted, lapply, Map, mapply, match, mget, order, paste, pmax,
    pmax.int, pmin, pmin.int, Position, rank, rbind, Reduce, rep.int,
    rownames, sapply, setdiff, sort, table, tapply, union, unique,
    unlist, unsplit

Loading required package: Biobase
Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
   

# Input data and preparations
## 1. The count table

In [5]:
CountData = read.table("./counts_for_deseq.txt",header=T,sep='\t',row.names = 1)

In [6]:
head(CountData)

,C20_2,C20_3,NC
ENSG00000000003,976,1443,1452
ENSG00000000005,49,56,30
ENSG00000000419,1904,1112,2507
ENSG00000000457,216,254,254
ENSG00000000460,596,670,729
ENSG00000000938,1,1,4


In [7]:
dim(CountData)

[1] 57736     3

In [8]:
# filtering low count reads
count = CountData[rowSums(CountData>10)>2,]
count = count + 1
print(dim(count))
head(count)

[1] 20184     3


,C20_2,C20_3,NC
ENSG00000000003,977,1444,1453
ENSG00000000005,50,57,31
ENSG00000000419,1905,1113,2508
ENSG00000000457,217,255,255
ENSG00000000460,597,671,730
ENSG00000001036,811,692,1024


## 2. The  Meta Data--sample Info

In [9]:
sample = c('C20_2','C20_3','NC')
condition =factor(c('C20_2','C20_3','NC'))
expTab = data.frame(condition,row.names= sample)

In [10]:
count = count[,sample]

In [11]:
head(count)

,C20_2,C20_3,NC
ENSG00000000003,977,1444,1453
ENSG00000000005,50,57,31
ENSG00000000419,1905,1113,2508
ENSG00000000457,217,255,255
ENSG00000000460,597,671,730
ENSG00000001036,811,692,1024


In [12]:
tail(count)

,C20_2,C20_3,NC
ENSG00000273451,115,194,98
ENSG00000273474,19,22,18
ENSG00000273478,23,40,16
ENSG00000273486,36,54,69
ENSG00000273488,23,12,35
ENSG00000273489,63,94,96


In [13]:
newCount = newCountDataSet(count,condition) # setup CountDataSet 
newCount2 = newCountDataSet(count[,c(1,3)],condition=c("C20_2","NC"))
newCount3 = newCountDataSet(count[,c(2,3)],condition=c("C20_3","NC"))

In [14]:
tail(newCount)

CountDataSet (storageMode: environment)
assayData: 1 features, 3 samples 
  element names: counts 
protocolData: none
phenoData
  sampleNames: C20_2 C20_3 NC
  varLabels: sizeFactor condition
  varMetadata: labelDescription
featureData: none
experimentData: use 'experimentData(object)'
Annotation:  

## 3. Normalization

In [15]:
cds = estimateSizeFactors(newCount)

In [16]:
sizeFactors(cds)

C20_2     C20_3        NC 
0.9182496 0.9499143 1.1422816

In [17]:
head(counts(cds,normalize=T))

,C20_2,C20_3,NC
ENSG00000000003,1063.981,1520.137,1272.016
ENSG00000000005,54.45142,60.00542,27.13867
ENSG00000000419,2074.599,1171.685,2195.606
ENSG00000000457,236.3192,268.4453,223.2374
ENSG00000000460,650.1500,706.3795,639.0718
ENSG00000001036,883.2021,728.4868,896.4515


## 4. Variance Estimation

### 4.1 Working without any replicates

In [18]:
cds2 = cds[,c("C20_2","NC")]
cds3 = cds[,c("C20_3","NC")]

In [19]:
pData(cds2)

,sizeFactor,condition
C20_2,0.9182496,C20_2
NC,1.142282,NC


In [20]:
cds2= estimateDispersions(cds2,method="blind",sharingMode="fit-only")
cds3= estimateDispersions(cds3,method="blind",sharingMode="fit-only")

In [21]:
str(fitInfo(cds2))

List of 5
 $ perGeneDispEsts: num [1:20184] 0.01502 0.20005 0.00115 -0.00265 -0.00138 ...
 $ dispFunc       :function (q)  
  ..- attr(*, "coefficients")= Named num [1:2] 0.0218 3.7896
  .. ..- attr(*, "names")= chr [1:2] "asymptDisp" "extraPois"
  ..- attr(*, "fitType")= chr "parametric"
 $ fittedDispEsts : num [1:20184] 0.025 0.1147 0.0236 0.0383 0.0277 ...
 $ df             : num 1
 $ sharingMode    : chr "fit-only"


### Calling Differential Expression

In [22]:
res2 = nbinomTest(cds2,"C20_2","NC")

In [23]:
res3 = nbinomTest(cds3,"C20_3","NC")

In [24]:
head(res2)

,id,baseMean,baseMeanA,baseMeanB,foldChange,log2FoldChange,pval,padj
1,ENSG00000000003,1167.998,1063.981,1272.016,1.195525,0.2576442,0.4393205,1
2,ENSG00000000005,40.79505,54.45142,27.13867,0.4984014,-1.00462,0.2174777,1
3,ENSG00000000419,2135.102,2074.599,2195.606,1.058328,0.08178629,0.7994844,1
4,ENSG00000000457,229.7783,236.3192,223.2374,0.9446437,-0.08215786,0.8581386,1
5,ENSG00000000460,644.6109,650.15,639.0718,0.9829606,-0.02479453,0.9487017,1
6,ENSG00000001036,889.8268,883.2021,896.4515,1.015001,0.02148184,0.9534498,1


In [25]:
res_sig2 = subset(res2,pval <=0.05)
res_sig_order2 = res_sig2[order(res_sig2$pval),]
write.csv(res_sig_order2,"C20_2_VS_NC.csv",row.names=F)

In [26]:
res_sig3 = subset(res3,pval <=0.05)
res_sig_order3 = res_sig3[order(res_sig3$pval),]
write.csv(res_sig_order3,"C20_3_VS_NC.csv",row.names=F)

In [27]:
#hist(res15$pval,breaks=100,col='skyblue',border="slateblue",main="")

In [25]:
res2$padj1 = p.adjust(res2$pval,method="BH")

In [26]:
subset(res2,padj!=padj1)

,id,baseMean,baseMeanA,baseMeanB,foldChange,log2FoldChange,pval,padj,padj1


In [27]:
res_sig_new2 = subset(res2, padj <=0.1)

In [28]:
head(res_sig_new2)

,id,baseMean,baseMeanA,baseMeanB,foldChange,log2FoldChange,pval,padj,padj1
64,ENSG00000005075,1073.155,560.4572,1585.853,2.829571,1.500583,1.096168e-05,0.004355871,0.004355871
746,ENSG00000064300,175.1583,67.47251,282.8441,4.19199,2.067635,1.952875e-05,0.006943327,0.006943327
791,ENSG00000065518,1822.558,997.9402,2647.175,2.652639,1.407428,1.760383e-05,0.00648653,0.00648653
1552,ENSG00000088986,4864.45,2990.556,6738.344,2.253208,1.17198,0.000193716,0.04176435,0.04176435
1645,ENSG00000091136,1683.041,1057.795,2308.288,2.18217,1.125763,0.0005769427,0.09993437,0.09993437
2382,ENSG00000103275,3616.238,2139.532,5092.944,2.380402,1.251205,8.088253e-05,0.02185554,0.02185554


In [52]:
dim(res_sig_new2)

[1] 117   9

In [28]:
res_sig_order = res_sig[order(res_sig$padj1),]

In [29]:
head(res_sig_order)

,id,baseMean,baseMeanA,baseMeanB,foldChange,log2FoldChange,pval,padj,padj1
15597,ENSMUSG00000076258.1,9878.281,19068.2,688.3652,0.03610017,-4.79185,2.975773e-27,5.948272e-23,5.948272e-23
19356,ENSMUSG00000098973.1,1711.912,3318.01,105.8139,0.03189076,-4.970718,1.700659e-26,1.699724e-22,1.699724e-22
19302,ENSMUSG00000098178.1,51722.59,98546.12,4899.067,0.04971344,-4.33022,7.912087e-24,5.271824e-20,5.271824e-20
10582,ENSMUSG00000041351.11,263.8229,8.927954,518.7179,58.10043,5.860477,5.418195e-21,2.707608e-17,2.707608e-17
15497,ENSMUSG00000075014.1,646.021,1239.71,52.33185,0.04221298,-4.56617,1.41619e-20,5.661646e-17,5.661646e-17
15599,ENSMUSG00000076281.1,363.9621,707.2215,20.70271,0.02927331,-5.09427,1.888503e-20,6.291546e-17,6.291546e-17


### Saving Results

In [30]:
write.csv(res_sig_order,file="WWR2_VS_KOR2.csv",row.names=F)

## RScripts for generating multi-comparsion files

In [23]:

# this script used for no biological replicates RNA-seq only. If you experiment design do have replicates, please refer to DESeq2
# after setting up CountDataSet, run the functions below.

deseq = function(rawCount,sample1,sample2,level1,level2,p=0.05,padj=0.1) {
    #
    #rawCount: accept the raw counts which you want to compare
    #sample1, sample2: column names your want to compare
    #level1, level2: levels or factors which sample1 or sampel 2 correspondes
    #p: pvalue cutoff. Default:0.05
    #padj: padj value cutoff. You can specify other values. Default:0.1
  
    newCount = newCountDataSet(rawCount[,c(sample1,sample2)],condition=c(level1,level2))
    cds = estimateSizeFactors(newCount)
    cds = estimateDispersions(cds,method="blind",sharingMode="fit-only")
    res = nbinomTest(cds,level1,level2)
    res_sig = res[res$pval<p,]
    res_ordered = res_sig[order(res_sig$pval),]
    write.csv(res_ordered,file=paste(level1,"_VS_",level2,".csv",sep=""),row.names=F)
}


In [22]:
head(count)

,W_W_R2.count,W_W_R3.count,W_H_R2.count,W_H_R3.count,KO_R2.count,KO_R3.count
ENSMUSG00000000001.4,4883,1072,3324,969,6262,912
ENSMUSG00000000003.10,111,54,187,70,189,96
ENSMUSG00000000028.9,1471,917,1672,1093,1568,881
ENSMUSG00000000031.10,449,376,547,706,376,297
ENSMUSG00000000037.11,109,15,71,19,114,13
ENSMUSG00000000049.6,24,18,28,24,33,9


# Rsrcipt for GO

In [27]:
library(org.Hs.eg.db) #org.Hs.eg.db for mouse

Loading required package: AnnotationDbi
Loading required package: stats4
Loading required package: GenomeInfoDb
Loading required package: S4Vectors
Loading required package: IRanges
Loading required package: DBI



In [40]:
annotation = function(res, sample1,sample2) {
    
    tmp=select(org.Hs.eg.db, keys=res$id, columns=c("ENTREZID","SYMBOL","GENENAME"), keytype="ENSEMBL")

    res=merge(tmp, res,by.x='ENSEMBL',by.y='id',all=TRUE)
    #go
    tmp=select(org.Hs.eg.db, keys=res$ENSEMBL, columns='GO', keytype='ENSEMBL')
    ensembl_go=unlist(tapply(tmp[,2],as.factor(tmp[,1]),function(x) paste(x,collapse ='|'),simplify =F))
    #为res加入go注释，
    res$go=ensembl_go[res$ENSEMBL]#为res加入一列go
    #写入all——data
    #all_res=res
    #write.csv(res, file=paste(sample1,"_vs_",sample2,".annotated.csv",sep=""),row.names=T)
    uniq=na.omit(res)#删除无效基因
    sort_uniq=uniq[order(uniq$pval),]#按照矫正p值排序
    #写入排序后的all_data
    #write.csv(res,file='c20_2vsNC_data.csv',row.names =F)
    #标记上下调基因
    sort_uniq$up_down=ifelse(sort_uniq$baseMeanA>sort_uniq$baseMeanB,'up','down')
    #交换上下调基因列位置
    #final_res=sort_uniq[,c(12,1:11)]
    #final_res = sort_uniq
    #写出最后数据
    write.csv(sort_uniq,file=paste(sample1,"_vs_",sample2,".final_annotated.csv",sep=""),row.names =F)
    #然后挑选出padj值小于0.1的数据来做富集
    tmp=select(org.Hs.eg.db, keys=sort_uniq[sort_uniq$padj<0.1,1], columns='ENTREZID', keytype='ENSEMBL')
    diff_ENTREZID=tmp$ENTREZID
    require(DOSE)
    require(clusterProfiler)
    diff_ENTREZID=na.omit(diff_ENTREZID)
    ego <- enrichGO(gene=diff_ENTREZID,organism='human',ont='BP',pvalueCutoff=0.05,readable=TRUE)
    #ekk <- enrichKEGG(gene=diff_ENTREZID, organism='human',pvalueCutoff=0.05,readable=TRUE)
    #write.csv(summary(ekk),'KEGG-enrich.csv',row.names =F)
    write.csv(summary(ego),'GO-enrich.csv',row.names =F)
    return(c(ego,ekk))
    }

In [41]:
results = annotation(res_sig_order2, 'C20_2','NC')

Warning message:
In is.na(object): is.na()不适用于类别为'NULL'的非串列或非矢量

ERROR: Error in provideDimnames(x, sep = sep, base = base): 'dimnames'只能用于陈列


In [31]:
help(enrichGO)

enrichGO {clusterProfiler},R Documentation
gene,a vector of entrez gene id.
organism,"One of ""anopheles"", ""arabidopsis"", ""bovine"", ""canine"", ""celegans"", ""chicken"", ""chimp"", ""coelicolor"", ""ecolik12"",""ecsakai"", ""fly"", ""gondii"",""human"", ""malaria"", ""mouse"", ""pig"", ""rat"",""rhesus"", ""xenopus"", ""yeast"" and ""zebrafish""."
ont,"One of ""MF"", ""BP"", and ""CC"" subontologies."
pvalueCutoff,Cutoff value of pvalue.
pAdjustMethod,"one of ""holm"", ""hochberg"", ""hommel"", ""bonferroni"", ""BH"", ""BY"", ""fdr"", ""none"""
universe,background genes
qvalueCutoff,qvalue cutoff
minGSSize,minimal size of genes annotated by Ontology term for testing.
readable,whether mapping gene ID to gene Name


In [42]:
    tmp=select(org.Hs.eg.db, keys=res_sig_order2$id, columns=c("ENTREZID","SYMBOL","GENENAME"), keytype="ENSEMBL")

    res=merge(tmp,res_sig_order2,by.x='ENSEMBL',by.y='id',all=TRUE)
    #go
    tmp=select(org.Hs.eg.db, keys=res$ENSEMBL, columns='GO', keytype='ENSEMBL')
    ensembl_go=unlist(tapply(tmp[,2],as.factor(tmp[,1]),function(x) paste(x,collapse ='|'),simplify =F))
    #为res加入go注释，
    res_sig_order2$go=ensembl_go[res$ENSEMBL]#为res加入一列go
    #写入all——data
    #all_res=res
    #write.csv(res, file=paste(sample1,"_vs_",sample2,".annotated.csv",sep=""),row.names=T)
    uniq=na.omit(res_sig_order2)#删除无效基因
    sort_uniq=uniq[order(uniq$pval),]#按照矫正p值排序
    #写入排序后的all_data
    #write.csv(res,file='c20_2vsNC_data.csv',row.names =F)
    #标记上下调基因


Warning message:
In is.na(object): is.na()不适用于类别为'NULL'的非串列或非矢量

ERROR: Error in provideDimnames(x, sep = sep, base = base): 'dimnames'只能用于陈列
